In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
import time

# ==========================================
# 설정 구간
# ==========================================
SEARCH_KEYWORD = "인공지능"  # 검색할 키워드 입력
MAX_ARTICLES = 5          # 수집할 기사 최대 개수 (네이버 뉴스 호스팅 링크 기준)
# ==========================================

def get_news_urls(keyword, limit=5):
    """
    네이버 검색 결과에서 '네이버 뉴스' 플랫폼에 호스팅된 기사 URL만 추출합니다.
    """
    urls = []
    page = 1  # start 파라미터 (1, 11, 21...)
    
    print(f"Checking keywords... '{keyword}'")
    
    while len(urls) < limit:
        # 네이버 뉴스 검색 URL
        search_url = f"https://search.naver.com/search.naver?where=news&query={keyword}&start={page}"
        print(search_url)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
        }
        
        try:
            response = requests.get(search_url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 검색 결과 중 '네이버뉴스' 아이콘이 있는 링크 찾기
            # div.info_group > a.info 클래스를 가진 태그가 네이버 뉴스 링크
            links = soup.select('div.info_group > a.info')
            
            found_current_page = False
            for link in links:
                if '네이버뉴스' in link.text and 'news.naver.com' in link['href']:
                    url = link['href']
                    if url not in urls:
                        urls.append(url)
                        print(f"기사 발견: {url}")
                        found_current_page = True
                    
                    if len(urls) >= limit:
                        break
            
            if not found_current_page:
                print("더 이상 새로운 관련 기사를 찾을 수 없습니다.")
                break
                
            page += 10 # 다음 페이지로 (네이버 검색은 10 단위)
            time.sleep(0.5)
            
        except Exception as e:
            print(f"검색 중 에러 발생: {e}")
            break
            
    return urls[:limit]

def get_news_ids(url):
    """
    URL에서 언론사 ID(oid)와 기사 ID(aid)를 추출합니다.
    """
    # URL 패턴 매칭 (PC 및 모바일 URL 모두 대응)
    match = re.search(r'(?:oid=|article\/)(\d+)(?:&aid=|\/)(\d+)', url)
    if match:
        return match.group(1), match.group(2)
    return None, None

def get_naver_comments(url):
    """
    특정 뉴스 기사의 댓글을 수집합니다.
    """
    oid, aid = get_news_ids(url)
    if not oid or not aid:
        print(f"건너뜀: 올바른 네이버 뉴스 URL이 아닙니다. ({url})")
        return []

    # 네이버 댓글 API 엔드포인트
    api_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'Referer': url
    }

    comments = []
    page = 1
    
    print(f"댓글 수집 시작: {url}")

    while True:
        # API 파라미터 설정
        params = {
            'ticket': 'news',          # 일반 뉴스: news
            'templateId': 'default_society',
            'pool': 'cbox5',
            'lang': 'ko',
            'country': 'KR',
            'objectId': f'news{oid},{aid}',
            'pageSize': 100,
            'page': page,
            'sort': 'favorite',        # favorite(순공감순), new(최신순)
            'initialize': 'true' if page == 1 else 'false',
            'useAltSort': 'true',
            'replyPageSize': 20
        }

        try:
            response = requests.get(api_url, headers=headers, params=params)
            content = response.text
            
            # JSONP or JSON parsing
            if '(' in content and ')' in content:
                start = content.find('(') + 1
                end = content.rfind(')')
                json_data = json.loads(content[start:end])
            else:
                json_data = json.loads(content)

            if not json_data.get('success'):
                # 댓글 기능이 없는 기사이거나 오류 발생 시
                break
                
            result = json_data.get('result', {})
            comment_list = result.get('commentList', [])
            
            if not comment_list:
                break

            for item in comment_list:
                comments.append({
                    '기사URL': url,
                    '작성자': item['userName'],
                    '내용': item['contents'],
                    '날짜': item['regTime'],
                    '공감수': item['sympathyCount'],
                    '비공감수': item['antipathyCount']
                })
            
            # 페이징 종료 조건
            total_count = result.get('count', {}).get('comment', 0)
            if len(comments) >= total_count:
                break
                
            page += 1
            time.sleep(0.3) # 딜레이

        except Exception as e:
            print(f"에러 발생: {e}")
            break
            
    print(f" -> {len(comments)}개 수집 완료")
    return comments


print(f"[{SEARCH_KEYWORD}] 키워드로 뉴스 기사를 검색합니다...")
news_urls = get_news_urls(SEARCH_KEYWORD, MAX_ARTICLES)

if news_urls:
    print(f"\n총 {len(news_urls)}개의 기사에서 댓글 수집을 진행합니다.\n" + "="*50)

    all_comments = []

    # 2. 각 URL별 댓글 수집
    for url in news_urls:
        article_comments = get_naver_comments(url)
        if article_comments:
            all_comments.extend(article_comments)
        time.sleep(1) # 기사 간 차단 방지 딜레이

    # 3. 결과 저장
    if all_comments:
        df = pd.DataFrame(all_comments)
        
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        file_name = f"naver_search_{SEARCH_KEYWORD}_{timestamp}.csv"
        
        # 컬럼 순서 지정
        cols = ['기사URL', '작성자', '내용', '날짜', '공감수', '비공감수']
        df = df[cols]
        
        df.to_csv(file_name, index=False, encoding='utf-8-sig')
        print(f"\n[완료] 총 {len(df)}개의 댓글이 '{file_name}' 파일로 저장되었습니다.")
    else:
        print("\n수집된 댓글이 없습니다.")
else:
    print("조건에 맞는 네이버 뉴스 기사를 찾지 못했습니다.")



[인공지능] 키워드로 뉴스 기사를 검색합니다...
Checking keywords... '인공지능'
https://search.naver.com/search.naver?where=news&query=인공지능&start=1


더 이상 새로운 관련 기사를 찾을 수 없습니다.
조건에 맞는 네이버 뉴스 기사를 찾지 못했습니다.
